In [1]:
from pyroomasync import ConnectedShoeBox, simulate
import matplotlib.pyplot as plt
import sys
import numpy as np
sys.path.append("../")

from IPython.display import display, Audio


def plot_chunk(mic_signals,
               labels,
               chunk_start=2,
               chunk_duration=1e-2,
               base_sr=32000):
    plt.figure(figsize=(10, 5))
    plt.locator_params(axis="x", nbins=12)
    
    n_mics = mic_signals.shape[0]
    start = int(base_sr*chunk_start)
    end = start + int(base_sr*chunk_duration)
    num_points = end - start
    x_axis = 1000*np.arange(num_points)/base_sr
    plt.title('Received signals')
    for i, mic_signal in enumerate(mic_signals):
        #plt.subplot(n_mics, 1, i + 1)
        plt.plot(x_axis,
                 mic_signal[start:end],
                 label=labels[i])
        #plt.title('Microphone {} signal'.format(i))
        plt.xlabel('Time (ms)')
    plt.legend(bbox_to_anchor=(1.05, 1))
    plt.tight_layout()

    plt.show()


In [5]:
from copy import copy
room_dims = [4, 5, 3]
sr_mic_1 = 32000
coords_mic_1 = [3, 2, 1]
coords_source = [2, 1, 1]
delay_mic_1 = 0

def simulate_and_plot(sr_mic_2, delay_mic_2, dist_mic_2):
    # Create room
    room = ConnectedShoeBox([4, 5, 3], fs=32000)
    coords_mic_2 = copy(coords_mic_1)
    # Change y coord of mic_2
    coords_mic_2[1] = coords_mic_2[1] + dist_mic_2
    
    # Add microphones with their sampling frequencies and latencies
    room.add_microphone(coords_mic_1, fs=sr_mic_1, delay=delay_mic_1)
    room.add_microphone(coords_mic_2, fs=sr_mic_2, delay=delay_mic_2)

    # Add a source
    room.add_source(coords_source, "data/vctk/p225_002.wav")

    # simulate and get the results recorded in the microphones
    simulation_results = simulate(room)

    labels=[
        f"sr={sr_mic_1}, delay={delay_mic_1}, coords={coords_mic_1}",
        f"sr={sr_mic_2}, delay={delay_mic_2}, coords={coords_mic_2}, "
    ]
    plot_chunk(simulation_results, labels)

## Interative simulation 

In [6]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [8]:
from ipywidgets import *

sr_slider = IntSlider(min=31800, max=32000, step=5, value=32000)
distance_slider = FloatSlider(min=0, max=1, step=0.03)
delay_slider = IntSlider(min=0, max=20, step=1)

def update(sr=sr_mic_1, delay_in_ms=0, dist=0):
    simulate_and_plot(sr, delay_in_ms/1000, dist)

interact(update,
         sr=sr_slider,
         delay_in_ms=delay_slider,
         dist=distance_slider)

interactive(children=(IntSlider(value=32000, description='sr', max=32000, min=31800, step=5), IntSlider(value=…

<function __main__.update(sr=32000, delay_in_ms=0, dist=0)>